In [1]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"

model_name="Local-Novel-LLM-project/Vecteus-v1"
#model_name="hatakeyama-llm-team/tanuki_inst_0515test"
model_name="kanhatakeyama/0516tanuki_lr5e5_epoch1"
#model_name="../X_merge/merged_models/0515_1_3_6"
model_name="../model/0516hf_t5"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


T5Tokenizer(name_or_path='../model/0516hf_t5', vocab_size=65000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<SEP>', 'pad_token': '<pad>', 'cls_token': '<CLS>', 'mask_token': '<MASK>', 'additional_special_tokens': ['<EOD>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<CLS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<SEP>", rstrip=False, lstrip=False, single_word

In [2]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16,
                                            )


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.04s/it]


In [3]:
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"
answer_template="\n\n### 応答:\n"

def gen_prompt(q):
    return f"{question_template}{q}{answer_template}"
    return question_template+q+answer_template

gen_prompt("こんにちは")

'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nこんにちは\n\n### 応答:\n'

In [4]:

pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=30, 
              #repetition_penalty=1.,
              temperature=0.1,
              #top_p=1.0,
              #top_k=0.,
              do_sample=True,
              )

question="""日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。"""
inp=gen_prompt(question)
r=pipe(inp)[0]["generated_text"][len(inp):]
print(r)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [24]:
prompt=inp
do_sample=True
#do_sample=False
temperature=0.1
max_new_token=30

question="""日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。"""
#question='以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。\n\n### 応答:\n'
#question='以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。\n\n### 応答:'
inp=gen_prompt(question)

inp='以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。\n\n### 応答:'
inp="こんにち"

def ask(prompt):
    input_ids = tokenizer([prompt]).input_ids
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        do_sample=do_sample,
        temperature=temperature,
        max_new_tokens=max_new_token,
        #repetition_penalty=1.0,
    )


    output_ids_=output_ids[0][len(input_ids[0]) :]
    output_text = tokenizer.decode(output_ids_, skip_special_tokens=True)
    return output_text
for i in range(3):
    print(i)
    print(ask(inp))

0

1

2



In [19]:

with torch.no_grad():
    input_ids = tokenizer.encode("こんにち", add_special_tokens=False, return_tensors="pt")
    output_ids = model.generate(
        input_ids.to(model.device), 
                                max_length=20, 
                                temperature=temperature,
                                do_sample=do_sample,
                                pad_token_id=tokenizer.pad_token_id)

tokenizer.decode(output_ids.tolist()[0])

'こんにちの社会は、多くの人が、自分の仕事や生活における成功を'

In [6]:
tokenizer

T5Tokenizer(name_or_path='../model/0516hf_t5', vocab_size=65000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<SEP>', 'pad_token': '<pad>', 'cls_token': '<CLS>', 'mask_token': '<MASK>', 'additional_special_tokens': ['<EOD>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<CLS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<SEP>", rstrip=False, lstrip=False, single_word

In [6]:
model_name="../model/0516hf_t5"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
print(tokenizer)
prompt="こんにち"

input_ids1 = tokenizer([prompt]).input_ids
input_ids2 = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
input_ids1,input_ids2

T5Tokenizer(name_or_path='../model/0516hf_t5', vocab_size=65000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<SEP>', 'pad_token': '<pad>', 'cls_token': '<CLS>', 'mask_token': '<MASK>', 'additional_special_tokens': ['<EOD>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<CLS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<SEP>", rstrip=False, lstrip=False, single_word

([[272, 48160]], tensor([[  272, 48160]]))

In [17]:
geniac_tokenizer = AutoTokenizer.from_pretrained("hatakeyama-llm-team/tanuki_inst_0515test")

prompt = 'こんにちは'
input_ids1 = geniac_tokenizer([prompt]).input_ids
input_ids2 = geniac_tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
print("hatakeyama-llm-team/tanuki_inst_0515test")
print(input_ids1, input_ids2)

print(tokenizer._add_eos_token)

hatakeyama-llm-team/tanuki_inst_0515test
[[272, 3124, 2]] tensor([[ 272, 3124]])


AttributeError: 'T5Tokenizer' object has no attribute 'add_eos_token'